In [7]:
generator1 = ("abcdefghijklmnopqrstuvwxyz", 7)
generator2 = ("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", 5)
cpu_threads = 6

In [8]:
from KrypBz.proj5.zdroje.crypt import crypt
from KrypBz.tools.file_tools import read_file_lines
import threading
import numpy

In [9]:
class Problem():
    def __init__(self, name, salt, fingerprint):
        self.name = name
        self.salt = salt
        self.fingerprint = fingerprint
        self.unlocked = False
        self.password = None

    def attempt_unlocking(self, passwords):
        for p in passwords:
            encrypted = crypt(p, self.salt)
            if encrypted.decode() == self.fingerprint:
                self.password = p
                self.unlocked = True
                break
        return self.unlocked

In [10]:
def unlock_users(users, passwords):
    for u in users:
        u.attempt_unlocking(passwords)

def unlock_users_parallel(users, passwords, threads):
    threads = []
    user_lists = numpy.array_split(users, threads)
    for i in range(cpu_threads):
        t = threading.Thread(target=unlock_users, args=(user_lists[i], passwords))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()

def unlock_file(path, passwords):
    lines = read_file_lines(path)
    print(lines)
    print(len(lines))
    users = []
    for l in lines:
        users.append(Problem(*(l.rstrip().split(':'))))
    unlock_users_parallel(users, passwords, cpu_threads)
    output = []
    for u in users:
        if u.unlocked:
            output.append(u)
    return output

def print_users(users):
    for u in users:
        print(u.name + ":\t" + u.password)

In [12]:
def generate_vocabulary():
    return []

def generate_all(alphabet, characters):
    a = []
    b = []
    for c in alphabet: #character
        a.append(c)
    for i in range(1, 4):
        for e in a: #element
            for c in alphabet:
                b.append(e+c)
        a = b
        b = []
    for e in a: #element
        for c in alphabet:
            b.append(e+c)
    return a + b

def generate_passwords():
    return generate_vocabulary() + generate_all(*generator1) + generate_all(*generator2)

passwords = generate_passwords()

MemoryError: 

In [ ]:
unlocked = unlock_file("./zdroje/shadow1.txt", passwords)
print_users(unlocked)

In [ ]:
unlocked = unlock_file("./zdroje/shadow2.txt", passwords)
print_users(unlocked)

In [ ]:
unlocked = unlock_file("./zdroje/shadow3.txt", passwords)
print_users(unlocked)

In [ ]:
unlocked = unlock_file("./zdroje/shadow4.txt", passwords)
print_users(unlocked)